In [6]:
using Pkg
Pkg.activate("/home/gribeill/GitHub/HiQuER/")
using HiQuER
using LRUCache
using BenchmarkTools
using OpenQASM
using StaticArrays
using QuantumClifford
using Match
using StatsBase
using JSON

  Activating project at `~/GitHub/HiQuER`


In [7]:
struct PauliBlock
    p :: PauliOperator
    ϕ :: Angle
    meas :: Bool
    hard :: Bool
end

PauliBlock(po::PauliOperator, ϕ::Angle, meas::Bool) = PauliBlock(po, ϕ, meas, (!meas && (ϕ.value == 1//8 || ϕ.value == -1//8)))

PauliBlock(xv::Vector{Bool}, zv::Vector{Bool}, t::Angle) = PauliBlock(PauliOperator(xv,zv), t, false)
PauliBlock(xv::Vector{Bool}, zv::Vector{Bool}, t::Angle, meas::Bool) = PauliBlock(PauliOperator(xv,zv), t, false, meas)

Base.length(pb::PauliBlock) = pb.p.nqubits

function rand_pauli_block(N)
    xvec = rand(Bool, N)
    zvec = rand(Bool, N)
    t = StatsBase.sample([Angle(1//4), Angle(-1//4), Angle(1//8), Angle(-1//8)], aweights([0.32, 0.32, 0.18, 0.18]))
    return PauliBlock(xvec, zvec, t)
end

rand_pauli_block (generic function with 1 method)

In [8]:
function commute(a::PauliBlock, b::PauliBlock)
    return comm(a.p, b.p) == 0x0
end

commute (generic function with 1 method)

In [9]:
function decompose_phase(t::Angle)
    return @match t begin
        Angle(0)    => []
        Angle(3//8) => [Angle(1//4), Angle(1//8)]
        Angle(5//8) => [Angle(1//2), Angle(1//8)]
        Angle(3//4) => [Angle(1//2), Angle(1//3)]
        Angle(7//8) => [Angle(1//2), Angle(1//4), Angle(1//8)]
        _           => [t]
    end
end

function decompose(pb::PauliBlock)
    tlist = decompose_phase(pb.ϕ)
    return [PauliBlock(pb.p, t, pb.meas) for t in tlist]
end

decompose (generic function with 1 method)

In [10]:
function y_free_equivalent(pb::PauliBlock)
    xvec = xbit(pb.p)
    zvec = zbit(pb.p)
    
    y_idx = findall(xvec .& zvec)
    
    if length(y_idx) == 0
        return PauliBlock(PauliOperator(xvec, zvec), pb.ϕ)
    end     
    
    zvec[y_idx] .= false
    
    left_ops = Vector{PauliBlock}()
    right_ops = Vector{PauliBlock}()
    
    if length(y_idx) % 2 == 0
        xx = zeros(Bool, length(xvec))
        zz = zeros(Bool, length(zvec))
        zz[popfirst!(y_idx)] = true
        push!(left_ops, PauliBlock(xx, zz, Angle(1//4)))
        push!(right_ops, PauliBlock(xx,zz,Angle(-1//4)))
    end
    
    xx = zeros(Bool, length(xvec))
    zz = zeros(Bool, length(zvec))
    zz[y_idx] .= true
    
    push!(left_ops, PauliBlock(xx, zz, Angle(1//4)))
    push!(right_ops, PauliBlock(xx,zz,Angle(-1//4)))
    
    return cat(left_ops, PauliBlock(xvec, zvec, pb.ϕ), right_ops, dims=1)
end

y_free_equivalent (generic function with 1 method)

In [11]:
mutable struct PauliCircuit
    ops   :: Vector{PauliBlock}
    size  :: Int
end

PauliCircuit() = PauliCircuit(Vector{PauliBlock}(), 0)

function Base.push!(pc::PauliCircuit, pb::PauliBlock)
    if pc.size == 0
        push!(pc.ops, pb)
        pc.size = pb.p.nqubits;
    else
        @assert length(pb) == pc.size "Wrong number of qubits!"
        push!(pc.ops, pb)
    end
end

function Base.append!(pc::PauliCircuit, pb::Vector{PauliBlock})
    for p in pb
        push!(pc, p)
    end
end

function Base.length(pc::PauliCircuit)
    return length(pc.ops)
end

function has_measurements(pc::PauliCircuit)
    return any([pb.meas for pb in pc.ops])
end

function get_hard_gates(pc::PauliCircuit)
    return findall(x -> x.hard, pc.ops)
end

function get_easy_gates(pc::PauliCircuit)
    return findall(x -> !x.hard, pc.ops)
end

function to_dict(pb::PauliBlock)
    d = Dict()
    if pb.meas
        d["pi*"] = pb.ϕ > 0 ? "M" : "-M"
    else
        d["pi*"] = string(numerator(pb.ϕ.value), "/", denominator(pb.ϕ.value))
    end
    for idx = 0:length(pb.p.nqubits)-1
        if pb.p[idx+1] == (true, false)
            d["q$idx"] = "X"
        elseif pb.p[idx+1] == (false, true)
            d["q$idx"] = "Z"
        elseif pb.p[idx+1] == (true, true)
            d["q$idx"] = "Y"
        else
            nothing
        end
    end
    return d
end
        

function to_dict(pc::PauliCircuit)
    d = Dict()
    d["layers"] = Dict()
    for (idx, pb) in enumerate(pc.ops)
        d["layers"][idx] = to_dict(pb)
    end
    return d
end

to_dict (generic function with 2 methods)

In [13]:
function decompose!(pc::PauliCircuit)
    new_ops = Vector{PauliBlock}()
    for op in pc.ops
        append!(new_ops, decompose(op))
    end
end

function to_y_free(pc::PauliCircuit)
    pc = PauliCircuit()
    for op in pc.ops
        append!(pc, y_free_equivalent(op))
    end
end     

to_y_free (generic function with 1 method)

In [14]:
function swap_commuting!(pc::PauliCircuit, index::Int)
    @inbounds pc.ops[index+1], pc.ops[index] = pc.ops[index], pc.ops[index+1]
end

function swap_anticommuting!(pc::PauliCircuit, index::Int)
    next = index+1
    @inbounds p_new = pc.ops[index].p * pc.ops[next].p
    if p_new.phase == 0x1
        pb_new = PauliBlock(p_new, pc.ops[next].ϕ * -1.0, pc.ops[next].meas)
    else #p_new.phase == 0x3
        pb_new = PauliBlock(p_new, pc.ops[next].ϕ, pc.ops[next].meas)
    #else
    #    throw(DomainError(nothing))
    end
    
    @inbounds pc.ops[next] = pc.ops[index]
    @inbounds pc.ops[index] = pb_new
end

function swap_adjacent!(pc::PauliCircuit, index::Int)
    next = index+1
    if next > length(pc.ops)
        error("Nothing to commute with!")
    end
    
    if (pc.ops[index].hard)
        error("First operand must be π/4 rotation")
    end
    
    if commute(pc.ops[index], pc.ops[next])
        #println("commute!")
        swap_commuting!(pc, index)
    else
        #println("anticommute!")
        swap_anticommuting!(pc, index)
    end
end

function swap!(v::Vector{Int}, idx::Int)
    @inbounds v[idx], v[idx+1] = v[idx+1], v[idx]
end

function apply_litinski!(pc::PauliCircuit)
    decompose!(pc)
    #Move all pi/4 to after all pi/8
    iter = 1
    
    last_hard  = findlast(x->x.hard, pc.ops)
    first_easy = findfirst(x->!x.hard, pc.ops)
        
    while true
        
        if last_hard < first_easy
            break #we're done
        end 
        for idx in (last_hard-1):-1:first_easy
            if @inbounds !pc.ops[idx].hard && pc.ops[idx+1].hard 
                if idx+1 == last_hard
                    last_hard -= 1
                end
                if idx == first_easy
                    first_easy += 1
                end
                swap_adjacent!(pc, idx)
            end
            if idx < first_easy
                break
            end
        end
        
    end
    #println(iter)
end  


apply_litinski! (generic function with 1 method)

In [33]:
mutable struct LayeredPauliCircuit
    layers :: Vector{Vector{PauliBlock}}
    easy_ops :: Vector{PauliBlock}
end

LayeredPauliCircuit() = LayeredPauliCircuit(Vector{Vector{PauliBlock}}(), Vector{PauliBlock}())

function LayeredPauliCircuit(pc::PauliCircuit)
    layers = Vector{Vector{PauliBlock}}()
    easy_ops = Vector{PauliBlock}()
    for op in pc.ops
        if op.hard
            push!(layers, [op])
        else
            push!(easy_ops, op)
        end
    end
    return LayeredPauliCircuit(layers, easy_ops)
end

function reduce_t_depth!(lpc::LayeredPauliCircuit)
    changed = true
    N = length(lpc.layers)
    while changed
        changed = false
        for idx = 1:N-1
            if idx+1 == length(lpc.layers)
                break
            end
            
            #items from layer idx+1 that commute with all in layer idx
            commuters = [jdx for (jdx, next_op) in enumerate(lpc.layers[idx+1]) if all([commute(next_op, first_op) for first_op in lpc.layers[idx]])]
            
            #swap the commuters from idx+1 to idx
            for cidx in reverse(commuters)
                push!(lpc.layers[idx], popat!(lpc.layers[idx+1], cidx))
                changed = true
            end
        end
    end
    first_empty = findfirst(x->length(x) == 0, lpc.layers)
    lpc.layers = lpc.layers[1:first_empty-1]
end

function to_dict(lpc::LayeredPauliCircuit, n::Int)
    t_count = sum([length(ly) for ly in lpc.layers])
    t_depth = length(lpc.layers)
    d = Dict()
    d["T count"] = t_count
    d["T depth"] = t_depth
    d["T layers"] = Dict()
    d["n"] = n
    for (idx, lyr) in enumerate(lpc.layers)
        d["T layers"][idx-1] = Dict()
        for (idx, op) in enumerate(lyr)
            d["T layers"][idx-1] = to_dict(op)
        end
    end
    d["pi/4 rotations and measurements"] = Dict()
    for (idx, op) in enumerate(lpc.easy_ops)
        d["pi/4 rotations and measurements"][idx-1] = to_dict(op)
    end
    return d
end 


to_dict (generic function with 3 methods)

In [16]:
function test_litinski(fn, N, jmax)
    pc = PauliCircuit()
    for j=1:jmax
        pb = rand_pauli_block(N)
        push!(pc, rand_pauli_block(N))
    end
    
    fn(pc)
end

test_litinski (generic function with 1 method)

In [17]:
@time test_litinski(apply_litinski!, 8, 4500)

  0.583042 seconds (7.18 M allocations: 297.034 MiB, 10.59% gc time, 15.35% compilation time)


In [18]:
function gate_to_pauli(gate::GG, idx::Union{Int, Tuple{Int, Int}}, N::Int) where GG<:HiQuER.AbstractGate
    pb = Vector{PauliBlock}()
    if gate == CNOT
        c = idx[1]
        t = idx[2]
        xv = zeros(Bool, N)
        zv = zeros(Bool, N)
        zv[c] = true
        xv[t] = true
        push!(pb, PauliBlock(xv, zv, Angle(1//4)))
        xv[t] = false
        push!(pb, PauliBlock(xv, zv, Angle(1//4)))
        zv[c] = false
        xv[t] = true
        push!(pb, PauliBlock(xv, zv, Angle(1//4)))
        return pb;
    end
    if gate ==  S 
        paulis = [(0, 1, Angle(1//4))]
    elseif gate == S'
        paulis = [(0, 1, Angle(-1//4))]
    elseif gate == H
        paulis = [(0, 1, Angle(1//4)), (1, 0, Angle(1//4)), (0, 1, Angle(1//4))]
    elseif gate == X180
        paulis = [(1, 0, Angle(1//2))]
    elseif gate == Y180
        paulis = [(1, 1, Angle(1//2))]
    elseif gate == Z180 
        paulis = [(0, 1, Angle(1//2))]
    elseif gate == T
        paulis = [(0, 1, Angle(1//8))]
    else
        error(gate)
    end
    for p in paulis
        xv = zeros(Bool, N)
        zv = zeros(Bool, N)
        xv[idx[1]] = Bool(p[1])
        zv[idx[1]] = Bool(p[2])
        push!(pb, PauliBlock(xv, zv, p[3]))
    end
    return pb
end      

gate_to_pauli (generic function with 1 method)

In [19]:
function circuit_to_pauli(c::Circuit, N::Int)
    pc = PauliCircuit()
    for sl in c.slices
        for (id, gate) in sl.gates
            #println(id)
            if length(id.id) == 1
                idx = id.id[1]
            else
                idx = (id.id[1], id[2])
            end
            #println(idx, gate)
            append!(pc, gate_to_pauli(gate, idx, N))
        end
    end
    return pc
end

circuit_to_pauli (generic function with 1 method)

In [53]:
function qasm_litinski(fname, N)
    infodict = Dict()
    circ = load_qasm_circuit(fname)
    pc = circuit_to_pauli(circ, N)
    infodict["1. Circuit as Pauli rotations"] = to_dict(pc)
    apply_litinski!(pc)
    infodict["2. Circuit after the Litinski Transform"] = to_dict(pc)
    lpc = LayeredPauliCircuit(pc)
    reduce_t_depth!(lpc)
    infodict["3. Circuit after T depth reduction"] = to_dict(lpc, N)
    return infodict
end

qasm_litinski (generic function with 1 method)

In [61]:
using JSON

In [66]:
Nq = [2^n for n=3:10]

8-element Vector{Int64}:
    8
   16
   32
   64
  128
  256
  512
 1024

In [ ]:
Nq = [2^n for n=3:10]
times = []

for n in Nq
    println("Working on $n")
    t = @elapsed info = qasm_litinski("../resource_estimates/plasma/qasm_BBN/ct/plasma_hamsim_$n.qasm", n);
    open("../resource_estimates/plasma/qasm_BBN/lattice_surgery/plasma_hamsim_$n.qasm", "w") do io
        JSON.print(io, info, 4)
    end
    println("Took $(t) seconds!\n---------------")
    push!(times, t)
end

Working on 8
Took 1.8582797 seconds!
---------------
Working on 16
Took 13.7378253 seconds!
---------------
Working on 32
Took 67.0919832 seconds!
---------------
Working on 64
Took 326.8542037 seconds!
---------------
Working on 128
Took 2122.1728132 seconds!
---------------
Working on 256
Took 13812.3903217 seconds!
---------------
Working on 512


In [65]:
open("../resource_estimates/plasma/qasm_BBN/lattice_surgery/plasma_hamsim_8.qasm", "w") do io
    JSON.print(io, info, 4)
end

In [43]:
lpc = LayeredPauliCircuit(pc);

In [44]:
reduce_t_depth!(lpc)

990-element Vector{Vector{PauliBlock}}:
 [PauliBlock(- ___ZX___, π/8, false, true), PauliBlock(- XY___X__, π/8, false, true), PauliBlock(- _______X, π/8, false, true)]
 [PauliBlock(-i__XY__X_, π/8, false, true), PauliBlock(+iXX___X_X, π/8, false, true)]
 [PauliBlock(- ___ZX___, π/8, false, true), PauliBlock(- _Z_____X, π/8, false, true)]
 [PauliBlock(-i__XY__X_, π/8, false, true), PauliBlock(+iXX___X_X, π/8, false, true)]
 [PauliBlock(- ___ZX___, π/8, false, true), PauliBlock(- XY___X__, π/8, false, true)]
 [PauliBlock(+ __XXX_X_, π/8, false, true), PauliBlock(+iXX___X_X, π/8, false, true)]
 [PauliBlock(- ___ZX___, π/8, false, true), PauliBlock(- _Z_____X, π/8, false, true)]
 [PauliBlock(+ __XXX_X_, π/8, false, true), PauliBlock(+iXX___X_X, π/8, false, true)]
 [PauliBlock(- ___ZX___, π/8, false, true), PauliBlock(- _Z_____X, π/8, false, true)]
 [PauliBlock(+ __XXX_X_, π/8, false, true), PauliBlock(- XY___X__, π/8, false, true)]
 [PauliBlock(-i__XY__X_, π/8, false, true), PauliBlock(- _

In [50]:
@time to_dict(lpc, 8)

  0.009601 seconds (156.21 k allocations: 9.035 MiB)


Dict{Any, Any} with 5 entries:
  "pi/4 rotations and measurements" => Dict{Any, Any}(4986=>Dict{Any, Any}("pi*…
  "T count"                         => 1707
  "T layers"                        => Dict{Any, Any}(719=>Dict{Any, Any}(), 69…
  "T depth"                         => 990
  "n"                               => 8